# Dynamic Matching

In this notebook, I set out to implement the Dynamic Matching algorithm described in [Veldhuizen 2007](0712.1549.pdf), page 16.

The point of the dynamic matching, as I understand it, is to create a coarse version of the graph which can be animated faster than the entire graph. The accelerated layout of the coarse graph is to inform the finer graph before its layout calculations start.

The only thing missing at this point is the layout function that would accompany the corase layout, but maybe I can directly wire up the Vertex and the 

## Helper Structures


To make a dynamic matching, we first need a graph. 

#### Vertex
A graph, of course, consists of vertices ...

In [1]:
/**
 * class Vertex
 * 
 * defines: 
 *  - constructor(Data)
 *  - add_to(Graph)
 *  - remove_from(Graph)
 * see details below. 
 * 
 * This is the msot basic storage class of our data structure. 
 */
class Vertex {

  /** 
   * Vertex(Data)
   * 
   * Assigns data to the vertex, assigns an id to the vertex. Creates a property 
   * called edges, a set in which to hold incident edges. 
   */
  constructor(data){
    this.data = data;
    this.id = ++Vertex.id;
    
    this.edges = new Set();
    return this;
  }
  
  union(other){
    return new DMVertex(this, other);
  }
  
  /**
   * add_to(Graph)
   * 
   * Adds this vertex to a graph. 
   */
  add_to(graph){
    graph.add(this);
    return this;
  }
  
  /**
   * delete_from(Graph)
   * 
   * Removes this vertex from the graph.
   */ 
  delete_from(graph){
    graph.delete(this);
    return this;
  }
  
  toString(){
    return `Vertex#${this.id}`;
  }
}
Vertex.id = 0;

0

#### Edge
... and edges

In [2]:
/*
 * The edge represents a connection between two vertices, here called source and
 * target. 
 */
class Edge {
  
  /*
   * new Edge(Vertex, Vertex, Data)
   * - the constructor
   * 
   * Adds itself to the target and source vertex, and stores data passed into it. 
   * Maintains a count to allow for multiple edges. Assigns a pseudo random order.
   */
  constructor(source, target, data){
    console.assert(
      source instanceof Vertex, 
      `Edge::constructor requires a Vertex source.`
    );
    console.assert(
      target instanceof Vertex, 
      `Edge::constructor requires a Vertex target.`
    );
    
    this.source = source;
    this.target = target;
    this.data = data;
    
    this.source.edges.add(this);
    this.target.edges.add(this);
    
    this.count = 0;
    this.order = Math.random();
    this.id = ++Edge.id;
    
    return this;
  }
  
  /*
   * Adds this edge to the specified graph. 
   */
  add_to(graph){
    graph.add(this);
    if(this.graphs){
      this.graphs.add(graph);
    }else{
      this.graphs = new Set([graph]);
    }
    return this;
  }
  
  /*
   * Removes this vertex from the specified graph.
   */
  delete_from(graph){
    graph.delete(this);
    this.graphs.delete(graph);
    
    return this;
  }
  
  /*
   * Retuns true if the two edges share a vertex. 
   */
  shares_vertex(other){
    return this.source == other.source 
      || this.source == other.target
      || this.target == other.source
      || this.target == other.target;
  }
  
  toString(){
    return `Edge#${this.id}(${this.source.id},${this.target.id})`;
  }
}
Edge.id = 0;

0

#### Graph

In [3]:
/*
 * Graph class
 * 
 * All operations that don't naturally belong to Vertex or Edge
 * are issued here. 
 */
class Graph {
  
  /*
    constructor
    
    defines a set V and a set E for vertices and edges, respectively. 
  */
  constructor(){
    this.id = ++Graph.id; // anew should be a keyword
    this.V = new Set();
    this.E = new Set();
    
    return this;
  }
  
  /*
    add(vertex|edge)
    
    Adds to the graph a vertex or edge passed into this function. 
  */
  add(vertex_or_edge){
    if(vertex_or_edge instanceof Vertex){
      var vertex = vertex_or_edge;
      this.V.add(vertex);
    }
    
    if(vertex_or_edge instanceof Edge){
      var edge = vertex_or_edge;
      this.E.add(edge);
      
      if(!this.V.has(edge.source)){
        this.V.add(edge.source);
      }
      
      if(!this.V.has(edge.target)){
        this.V.add(edge.target);
      }
    }
    
    return vertex_or_edge;
  }
  
  
  /*
    delete(vertex|edge)
    
    Removes from the graph a vertex or edge passed into this function. 
    
    When a vertex is removed, its incident edges are removed, as well. 
    When an edge is removed, its references in its incident vertices are removed, 
    as well. 
  */
  delete(vertex_or_edge){
    if(typeof vertex_or_edge == "Vertex"){
      var vertex = vertex_or_edge;
      
      for(var edge in vertex.edges){
         edge.delete_from(this);
      }
      
      this.V.delete(vertex);
    }
    
    if(typeof vertex_or_edge == "Edge"){
      var edge = vertex_or_edge;
      edge.source.delete_from(this);
      edge.target.delete_from(this);
      this.E.delete(edge);
    }
    
    return vertex_or_edge;
  }
  
  toString(){
    return `Graph#${this.id}{|V|:${this.V.size},|E|:${this.E.size}}`;
  }
  
  get size(){
    return this.V.size + this.E.size;
  }
  
  get complexity(){
    return this.V.size / this.E.size;
  }
}
Graph.id = 0;

0

This will do, as it is close to how I actually use the graph in FourD.js.

Cool. Let's try it out: 


In [4]:
/*var v1 = new Vertex();
var v2 = new Vertex();
var g = new Graph();
g.add(v1)*/


In [5]:
/*g.add(new Edge(v1, v2))*/

### A Priority Queue
Now we can create a dynamic coarsening matching.
I borrewed a Priority Queue from geeksforgeeks:

In [6]:
// to store element and its priority
class QElement {
  constructor(element, priority)
  {
    this.element = element;
    this.priority = priority;
  }
}
 
// PriorityQueue class
class PriorityQueue {
 
  // An array is used to implement priority
  constructor(){
    this.items = [];
  }

  /* 
    enqueue function to add element to the queue as per priority
  */
  enqueue(element, priority){
    // creating object from queue element
    var qElement = new QElement(element, priority);
    var contain = false;

    // iterating through the entire
    // item array to add element at the
    // correct location of the Queue
    for (var i = 0; i < this.items.length; i++) {
      if (this.items[i].priority > qElement.priority) {
        // Once the correct location is found it is
        // enqueued
        this.items.splice(i, 0, qElement);
        contain = true;
        break;
      }
    }

    // if the element have the highest priority
    // it is added at the end of the queue
    if (!contain) {
      this.items.push(qElement);
    }
  }

  /*
    dequeue method to remove element from the queue
  */
  dequeue(){
    // return the dequeued element
    // and remove it.
    // if the queue is empty
    // returns Underflow
    if (this.empty()) throw "Underflow";
    return this.items.shift().element;
  }

  empty(){
    // return true if the queue is empty.
    return this.items.length == 0;
  }
}

## Dynamic Matching Structures

Now it's time for the dynamic matching. It's an intellectual whopper. The idea is that there's a series of increasingly coarser graphs, with vertices representing up to multiple vertices in the finer graphs. All the edges are ordered, and the basic unit of information is the edge, with its random order. 

"Our basic analysis tool is the edge graph G∗ = (E, S) whose vertices are
the edges of G, and e1Se2 when the edges share a vertex." - Veldhuizen 2007

### DMVertex

In [7]:
/**
 * class DMVertex
 * 
 * Encapsulates a vertex with a set union operation
 */
class DMVertex extends Vertex {
  
  /**
   * new DMVertex(vertex1, vertex2, ...)
   * 
   * 
   */
  constructor(){
    super(...arguments);
    this.finer = new Set([...arguments]);
    [...this.finer].map(v => {
      v.coarser = this;
    });
    
    return this;
  }

  /**
   * 
   */
  union(dmvertex){
    return new DMVertex(this, dmvertex);
  }

  has(vertex){
    return this.finer.has(vertex) 
      || [...this.finer].some(v => {
        return (v == vertex) || ((v instanceof DMVertex) && v.finer.has(vertex))
      });
  }
  
  toString(){
    return `DMVertex#${this.id}`;
  }
}
DMVertex.all = new Set();

Set {}

### DMEdge

In [8]:
/**
 * class DMEdge
 * 
 * Encapsulates an edge with a count for multiple edges across the 
 * same vertices. 
 */
class DMEdge extends Edge {
  constructor(edge){
    if(arguments[0] instanceof Edge){
      super(edge.source, edge.target);
      edge.coarser = this;
      this.finer = edge;
    }else{
      super(...arguments);
    }

    this.count = 0;
  }
  
  toString(){
    return `DMEdge#${this.id}(${this.source.id},${this.target.id})`;
  }
}

### Dynamic Matching

In [9]:
/**
 * class DynamicMatching
 * 
 * responsible for maintaining coarser and coarser versions of an
 * initially attached graph. 
 */
class DynamicMatching {
  /**
   * new DynamicMatching(base Graph|DynamicMatching, Integer)
   * 
   * - base specifies the base graph which is to be reduced, and to which this 
   *   dynamic matching should attach itself to, decorating add and delete in the base 
   *   object.
   * - n specifies how many levels of DynamicMatchings should be produced.
   * 
   * Instantiates the member variables id, finer, coarser, V (Map), E (Set) and pq. 
   */
  constructor(finer, n){
    this.id = ++DynamicMatching.id;
    
    this.finer = finer; // the base object
    finer.coarser = this; // a doubly linked list
    this.V = new Map(); // holds the vertices of this dynamic matching
    this.E = new Set(); // holds the edges of this dynamic matching
    this.pq = new PriorityQueue(); // ge
    
    this.m = new Map();
    
    if(n > 0){
      this.coarser = new DynamicMatching(this, --n);
    }

    var finer_add = finer.add.bind(finer);
    var finer_delete = finer.delete.bind(finer);

    var that = this;
    console.log(`${this} rewiring ${finer}'s add and delete.`);
    finer.add = (entity) => {
      console.assert(entity, `entity must exist to be added to ${finer} and ${that}`);
      finer_add(entity);
      if(entity instanceof Edge){
        that.add(entity);
      }
      
      return entity;
    }

    finer.delete = (entity) => {
      console.assert(entity, `entity must exist to be removed from ${that} and ${finer}`);
      finer_delete(entity);
      that.delete(this.get_corresponding(entity));

      return entity;
    }
    
    return this;
  }

  add(v_or_e){
    if(v_or_e instanceof Vertex){
      var vertex = v_or_e;
      this.add_vertex(vertex);
    }
    if(v_or_e instanceof Edge){
      var edge = v_or_e;
      this.add_edge(edge);
    }
    
    this.process_queue();
    return v_or_e;
  }

  /*
    Adds a new DMVertex for the supplied base vertex.
  */
  add_vertex(v){
    
    // "No action needed."
    /*
    if(!this.get_corresponding(v)){
      this.V.set(vertex, new DMVertex(v));
    }
    */
  }

  get_corresponding_edge(e){
    var e_prime = [...this.E].find(e_prime => {
      return e_prime.source.has(e.source) 
        || e_prime.target.has(e.target)
        || e_prime.source.has(e.target)
        || e_prime.target.has(e.source);
    });

    if(!e_prime){
      e_prime = new DMEdge(
        this.get_corresponding(e.source),
        this.get_corresponding(e.target)
      );
      this.E.add(e_prime);
    }

    return e_prime;
  }
  
  get_corresponding_vertex(v){
    var v_prime = this.V.get(v);
    if(!v_prime){
      v_prime = new DMVertex(v);
      this.V.set(v, v_prime);
    }
    return v_prime;
  }
  
  get_corresponding(v_or_e){
    if(v_or_e instanceof Vertex){
      return this.get_corresponding_vertex(v_or_e);
    }
    
    if(v_or_e instanceof Edge){
      return this.get_corresponding_edge(v_or_e);
    }
  }

  add_edge(e){
    // "Increase the count of (v1', v2') in E' possibly adding an edge..."
    var v1_prime = this.get_corresponding(e.source);
    var v2_prime = this.get_corresponding(e.target);
    
    var e_prime = this.get_corresponding(e);
    if(e_prime){
      e_prime.count++;
    }else{
      e_prime = new DMEdge(v1_prime, v2_prime);
      this.E.add(e_prime);
    }
    
    // "add e to the queue"
    this.pq.enqueue(e, e.order);
  }

  delete(v_or_e){
    //console.log(`Dynamic Matching ${this.id} deleting ${v_or_e}`);

    if(v_or_e instanceof Vertex){
      var vertex = v_or_e;
      return this.delete_vertex(vertex);
    }

    if(v_or_e instanceof Edge){
      var edge = v_or_e;
      return this.delete_edge(edge);
    }
    
    return v_or_e;
  }

  delete_vertex(v){
    [...v.edges].map(e => this.delete(this.get_corresponding(e)));
    this.V.delete(v);
  }

  delete_edge(e){
    // "If e is in the matching, then unmatch(e)"
    var e_prime = [...this.E].find(e_prime => {
      var v1_prime = e_prime.source;
      var v2_prime = e_prime.target;
      return v1_prime.has(e.source) 
        && v2_prime.has(e.target);
    });

    if(e_prime){
      this.unmatch(e);
      
      e_prime.count--;
      if(e_prime.count == 0){
        this.delete(e_prime);
      }

      console.log(`${e}>${e.source}>${e.source.edges.size} E`);
      if(e.source.edges.size == 0){
        this.delete(this.get_corresponding(e.source));
      }
      
      console.log(`${e}>${e.target}>${e.target.edges.size} E`);
      if(e.target.edges.size == 0){
        this.delete(this.get_corresponding(e.target));
      }
    }

    [...this.E]
    .filter(edge => this.depends(e, edge))
    .map(edge => this.pq.enqueue(edge, edge.order));
  }

  depends(e1, e2){
    // "e1 -> e2 === (e1 < e2) and e1 S e2"
    var priority = e1.order < e2.order;
    var share_vertex = e1.shares_vertex(e2);

    return priority && share_vertex;
  }
  
  match(e){
    console.assert(e instanceof Edge, `DM::match didn't receive a Edge`);
    console.assert(e.source instanceof Vertex, `DM::match received a Edge but it doesn't have a Vertex as source`);
    console.assert(e.target instanceof Vertex, `DM::match received a Edge but it doesn't have a Vertex as target`);
    
    console.log(`DM#${this.id} matching ${e}`);

    // For each edge e' where e -> e', if e' is matched then
    // unmatch(e').
    [...this.E]
    .filter(e2 => this.depends(e, e2) && this.get_corresponding(e2))
    .map(e2 => this.unmatch(e2));
    
    // Delete vertices v1_coarse and v2_coarse from the coarser graph.
    var v1_prime = this.get_corresponding(e.source);
    var v2_prime = this.get_corresponding(e.target);
    this.delete(v1_prime);
    this.delete(v2_prime);

    // Create a new vertex v1 U v2 in G'. 
    var v1_u_v2 = new DMVertex(e.source, e.target);
    this.add(v1_u_v2);

    // For all edges e = (v, v') in G incident on v1 or v2
    // (but not both), add a corresponding edge to or from v1 U v2
    // in G'.
    [...e.source.edges]
    .filter(edge => edge != e)
    .map(edge => {
      var other_vertex = new DMVertex(edge.source);
      var corresponding_edge = new DMEdge(other_vertex, v1_u_v2);
      this.add(other_vertex);
      this.add(corresponding_edge);
    });
    [...e.target.edges]
    .filter(edge => edge != e)
    .map(edge => {
      var other_vertex = new DMVertex(edge.target);
      var corresponding_edge = new DMEdge(v1_u_v2, other_vertex);
      this.add(other_vertex);
      this.add(corresponding_edge);
    });

    [...this.E]
    .filter(edge => this.depends(edge, e))
    .map(edge => this.pq.enqueue(edge, edge.order));
  }

  unmatch(e){
    console.log(`DM ${this.id} unmatching ${e}`);

    // "Delete any edges in G' incident on v1_u_v2."
    var v1_u_v2 = this.get_corresponding(e.source);
    console.assert(v1_u_v2, `v1_u_v2 not found`)
    console.assert(v1_u_v2 == this.get_corresponding(e.target), `sanity`);

    [...v1_u_v2.edges]
    .map(incident_edge => this.delete(incident_edge));

    
    // "Delete the vertex v1_u_v2 from G'"
    this.delete(v1_or_v2);
    
    // "Add new vertices v1_prime and v2_prime to G'"
    this.V.set(e.source, new DMVertex(e.source));
    this.V.set(e.target, new DMVertex(e.target));
    
    // "For each edge incident on v1 or v2 in G add a corresponding edge to G'"
    [...e.source.edges]
    .map(edge => this.add(new DMEdge(
      this.get_corresponding(e.source),
      this.get_corresponding(edge.source == e.source ? edge.target : edge.source)
    )));
    [...e.target.edges]
    .map(edge => this.add(new DMEdge(
      this.get_corresponding(e.target),
      this.get_corresponding(edge.target == e.target ? edge.source : edge.target)
    )));

    // "For each e' such that e -> e', add e' to the queue."
    [...this.E]
    .filter(edge => this.depends(e, edge))
    .map(edge => this.pq.enqueue(edge, edge.order));
  }

  match_equation(e){
    if(this.E.size == 0){
      console.log('because empty');
      return true;
    }
    
    // "... e is matched if and only if there is no edge e' element M
    // such that e' -> e."
    var m = [...this.E]
    .every(edge => !this.depends(edge, e));
    this.m.set(e, m);

    return m;
  }

  process_queue(){
    while(!this.pq.empty()){
      var e = this.pq.dequeue();
      console.assert(e instanceof Edge, `pq didn't return an Edge`);
      var m = this.match_equation(e);
      if(m != this.m.get(e)){
        if(m){
          this.match(e);
        }else{
          this.unmatch(e);
        }
      }
    }
  }

  toString(){
    return `DynamicMatching#${this.id}{|V|:${this.V.size},|E|:${this.E.size}}`;
  }
  
  get size(){
    return this.E.size + this.V.size;
  }
  
  get complexity(){
    return this.V.size / this.E.size;
  }
}
DynamicMatching.id = 0;

0

## Experiments

### The example graph from the paper.

In [10]:
var LEVELS = 2;

function coarser(base, level){
  if(level){
    return coarser(base.coarser, --level);
  }
  return base
}

var print_all = function(also){
  var str = ``;
  for(var l=0; l<=LEVELS+1; l++){
    str += `${coarser(graph, l).toString()}\n`;
  }

  if(also instanceof Function){
    for(var l=0; l<=LEVELS+1; l++){
      str += `${also(coarser(graph, l)).toString()}\n`;
    }
  }
  
  return str;
}

Vertex.id = 0;
Edge.id = 0;
Graph.id = 0;
DynamicMatching.id = 0;

var graph = new Graph();
var dm = new DynamicMatching(graph, LEVELS);

console_log = console.log;
console.log = function(){
  console_log(...[...arguments].map(arg => arg.toString()));
}

/*
graph_add = graph.add;
graph.add = (entity) => {
  console.assert(entity, `entity must be a value`);
  console.log(`${graph} adding ${graph_add(entity)} --> ${graph}`);
  
  console.log(print_all());
  
  return entity;
};

graph_delete = graph.delete;
graph.delete = (entity) => {
  console.assert(entity, `entity must be a value`);
  console.log(`${graph} removing ${graph_delete(entity)} --> ${graph}`);
  
  console.log(print_all());
  
  return entity;
}

for(var l=1; l<=LEVELS; l++){
  var dm = coarser(graph, l);
  
  dm_add = DynamicMatching.prototype.add.bind(dm);
  dm.add = (entity) => {
    console.assert(entity, `entity must be a value`);
    console.log(`${dm} adding ${dm_add(entity)} --> ${dm}`);
    return entity;
  }
  
  dm_delete = DynamicMatching.prototype.delete.bind(dm);
  dm.delete = (entity) => {
    console.assert(entity, `entity must be a value`);
    console.log(`${dm} removing ${dm_delete(entity)} --> ${dm}`);
    return entity;
  }
  
  var dm_match_equation = DynamicMatching.prototype.match_equation.bind(dm);
  dm.match_equation = (e) => {
    var m = dm_match_equation(e);
    console.log(`DM#${dm.id}'s match equation for ${e}' --> ${m}`);
    return m;
  }
}
*/

var v1 = new Vertex();
var v2 = new Vertex();
var v3 = new Vertex();
var v4 = new Vertex();
var v5 = new Vertex();
var v6 = new Vertex();
//console.log(v1);

graph.add(v1);
graph.add(v2);
graph.add(v3);
graph.add(v4);
graph.add(v5);
graph.add(v6);

var e1 = new Edge(v1, v2);
var e2 = new Edge(v1, v3);
var e3 = new Edge(v2, v4);
var e4 = new Edge(v3, v4);
var e5 = new Edge(v3, v5);
var e6 = new Edge(v4, v6);
var e7 = new Edge(v5, v6);

e1.order = 6;
e2.order = 2;
e3.order = 3;
e4.order = 7;
e5.order = 5;
e6.order = 1;
e7.order = 4;


graph.add(e6);
graph.add(e2);
graph.add(e3);
graph.add(e7);
graph.add(e5);
graph.add(e1);
graph.add(e4);

console.log(print_all());

DynamicMatching#3{|V|:0,|E|:0} rewiring DynamicMatching#2{|V|:0,|E|:0}'s add and delete.
DynamicMatching#2{|V|:0,|E|:0} rewiring DynamicMatching#1{|V|:0,|E|:0}'s add and delete.
DynamicMatching#1{|V|:0,|E|:0} rewiring Graph#1{|V|:0,|E|:0}'s add and delete.
Graph#1{|V|:6,|E|:7}
DynamicMatching#1{|V|:6,|E|:2}
DynamicMatching#2{|V|:6,|E|:2}
DynamicMatching#3{|V|:6,|E|:2}



### A Simple Random Graph

In [11]:
/**
 * element choice(iterable)
 *
 * Returns a pseudorandom element from the iterable by converting it 
 * to an array and choosing a random element. 
 */
var choice = function(iterable){
  return [...iterable][Math.floor(iterable.size*Math.random())];
}

/**
 * randomize_graph(graph, v, e)
 * 
 * adds the specified number of vertices (v) and edges (e) to the graph. 
 */
var randomize_graph = function(graph, v, e){
  for(var i=0; i<v; i++){
    graph.add(new Vertex());
  }
  
  for(var i=0; i<e; i++){
    graph.add(new Edge(choice(graph.V), choice(graph.V)));
  }
  
  return graph;
}

randomize_graph(graph, 100, 500);
console.log(print_all());

Graph#1{|V|:106,|E|:507}
DynamicMatching#1{|V|:106,|E|:50}
DynamicMatching#2{|V|:106,|E|:50}
DynamicMatching#3{|V|:106,|E|:50}



### Increasingly large random graphs

In [12]:
var MAX_SIZE = 10000;

var collect_all = function(also){
  var ret = [];
  
  for(var l=0; l<=LEVELS+1; l++){
    var obj = {};
    
    var g = coarser(graph, l);
    obj[`graph_V_size`] = g.V.size;
    obj[`graph_E_size`] = g.E.size;
    obj[`graph_size`] = g.size;
    obj[`graph_complexity`] = g.complexity;
    obj.level = l;
    
    ret.push(obj);
  }
  
  return ret;
}

var also = function(graph){
  graph_size = graph.size;
  
  try{
    coarser_size = graph.coarser ? graph.coarser.size : 1;
    finer_size = graph.finer ? graph.finer.size : 1;
  
    finer_ratio = graph_size / finer_size;
    coarser_ratio = graph_size / coarser_size;
  }catch(e){}
  
  return `
    ${graph}.size: ${graph.size}
    ${graph}.complexity: ${graph.complexity}
    ${graph}/${graph.coarser}: ${coarser_ratio}
    ${graph}/${graph.finer}: ${finer_ratio}
  `
}

var all = [];
for(var i=2; graph.size<MAX_SIZE; i*=2){
  randomize_graph(graph, i, i*2);
  console.log(graph.size)
  all = all.concat(collect_all(also));
}
console.log(all);

[ { graph_V_size: 108,
    graph_E_size: 511,
    graph_size: 619,
    graph_complexity: 0.21135029354207435,
    level: 0 },
  { graph_V_size: 106,
    graph_E_size: 50,
    graph_size: 156,
    graph_complexity: 2.12,
    level: 1 },
  { graph_V_size: 106,
    graph_E_size: 50,
    graph_size: 156,
    graph_complexity: 2.12,
    level: 2 },
  { graph_V_size: 106,
    graph_E_size: 50,
    graph_size: 156,
    graph_complexity: 2.12,
    level: 3 },
  { graph_V_size: 112,
    graph_E_size: 519,
    graph_size: 631,
    graph_complexity: 0.21579961464354527,
    level: 0 },
  { graph_V_size: 106,
    graph_E_size: 50,
    graph_size: 156,
    graph_complexity: 2.12,
    level: 1 },
  { graph_V_size: 106,
    graph_E_size: 50,
    graph_size: 156,
    graph_complexity: 2.12,
    level: 2 },
  { graph_V_size: 106,
    graph_E_size: 50,
    graph_size: 156,
    graph_complexity: 2.12,
    level: 3 },
  { graph_V_size: 120,
    graph_E_size: 535,
    graph_size: 655,
    graph_complexity:

## Conclusions



This algorithm seems to compress graphs by up to 30%. 